In [1]:
import shelly.shelly as shelly

shelly.save_yesterday_solar_production()
shelly.download_mains_data()

79.4kiB [00:00, 86.5kiB/s]


Download completed. File saved as /workspace/data/shelly/em_data.solar.csv
Yesterday's solar production: 
                           solar_Wh
timestamp_utc                      
2024-04-18 00:00:00+00:00   9466.41


79.6kiB [00:00, 85.8kiB/s]

Download completed. File saved as /workspace/data/shelly/em_data.main.csv


In [9]:
import pandas as pd

main_df = pd.read_csv(
    "/workspace/data/shelly/em_data.main.csv",
    sep=",",
    names=[
        "timestamp_utc",
        "main_consumed_energy(Wh)",
        "main_returned_energy(Wh)",
        "main_voltage_min(V)",
        "main_voltage_max(V)",
    ],
    dtype={
        "main_consumed_energy(Wh)": "Float64",
        "main_returned_energy(Wh)": "Float64",
        "main_voltage_min(V)": "Float64",
        "main_voltage_max(V)": "Float64",
    },
    parse_dates=["timestamp_utc"],
    header=0,
)
solar_df = pd.read_csv(
    "/workspace/data/shelly/em_data.solar.csv",
    sep=",",
    names=[
        "timestamp_utc",
        "solar_produced_energy(Wh)",
        "solar_returned_energy(Wh)",
        "solar_voltage_min(V)",
        "solar_voltage_max(V)",
    ],
    dtype={
        "solar_produced_energy(Wh)": "Float64",
        "solar_returned_energy(Wh)": "Float64",
        "solar_voltage_min(V)": "Float64",
        "solar_voltage_max(V)": "Float64",
    },
    parse_dates=["timestamp_utc"],
    header=0,
)

In [10]:
main_df.index = pd.to_datetime(main_df["timestamp_utc"]).dt.tz_localize("UTC")
main_df.drop(
    columns=["timestamp_utc", "main_voltage_min(V)", "main_voltage_max(V)"],
    inplace=True,
)
main_df.rename(columns={"main_consumed_energy(Wh)": "main_consumed_Wh"}, inplace=True)
main_df.rename(columns={"main_returned_energy(Wh)": "main_returned_Wh"}, inplace=True)
main_df.head()

,main_consumed_Wh,main_returned_Wh
timestamp_utc,,
2024-04-16 07:40:00+00:00,0.0,7.6
2024-04-16 07:50:00+00:00,20.3,22.5
2024-04-16 08:00:00+00:00,29.5,34.6
2024-04-16 08:10:00+00:00,0.0,64.0
2024-04-16 08:20:00+00:00,0.0,73.3


In [11]:
solar_df.index = pd.to_datetime(solar_df["timestamp_utc"]).dt.tz_localize("UTC")
solar_df.drop(
    columns=[
        "timestamp_utc",
        "solar_returned_energy(Wh)",
        "solar_voltage_min(V)",
        "solar_voltage_max(V)",
    ],
    inplace=True,
)
solar_df.rename(columns={"solar_produced_energy(Wh)": "solar_Wh"}, inplace=True)
solar_df.head()

,solar_Wh
timestamp_utc,
2024-04-19 00:00:00+00:00,0.08
2024-04-19 00:01:00+00:00,0.08
2024-04-19 00:02:00+00:00,0.08
2024-04-19 00:03:00+00:00,0.08
2024-04-19 00:04:00+00:00,0.08


In [14]:
shelly_df = pd.concat([main_df, solar_df], axis=1)
shelly_df = shelly_df.resample("15min").sum()
shelly_df["main_Wh"] = shelly_df["main_consumed_Wh"] - shelly_df["main_returned_Wh"]
shelly_df["consumed_Wh"] = shelly_df["main_Wh"] + shelly_df["solar_Wh"]
shelly_df = shelly_df[["main_Wh", "solar_Wh", "consumed_Wh"]]
shelly_df = shelly_df / 1000
shelly_df.rename(
    columns={
        "main_Wh": "main_kWh",
        "solar_Wh": "solar_kWh",
        "consumed_Wh": "consumed_kWh",
    },
    inplace=True,
)
shelly_df["main_kWh"] = shelly_df["main_kWh"].round(6)
shelly_df["solar_kWh"] = shelly_df["solar_kWh"].round(6)
shelly_df["consumed_kWh"] = shelly_df["consumed_kWh"].round(6)
shelly_df.to_csv("/workspace/data/shelly_consumption_history.csv")
shelly_df.tail()

,main_kWh,solar_kWh,consumed_kWh
timestamp_utc,,,
2024-04-19 03:30:00+00:00,0.05579,0.0012,0.05699
2024-04-19 03:45:00+00:00,0.0412,0.0012,0.0424
2024-04-19 04:00:00+00:00,0.05446,0.0012,0.05566
2024-04-19 04:15:00+00:00,0.08269,0.0012,0.08389
2024-04-19 04:30:00+00:00,0.06564,0.00112,0.06676


In [6]:
import providers.repsol as repsol

repsol_prices_df = repsol.get_latest_prices()

In [7]:
costs_df = shelly_df.join(repsol_prices_df, how="inner")
costs_df["bought_€"] = (costs_df["€/kWh"] * costs_df["main_kWh"]).clip(lower=0)
costs_df["lost_€"] = -(costs_df["€/kWh"] * costs_df["main_kWh"]).clip(upper=0)
costs_df["solar_production_€"] = costs_df["€/kWh"] * costs_df["solar_kWh"]
costs_df["solar_consumtion_€"] = costs_df["solar_production_€"] - costs_df["lost_€"]
costs_df["consumed_€"] = costs_df["€/kWh"] * costs_df["consumed_kWh"]
costs_df[
    [
        "consumed_€",
        "bought_€",
        "solar_consumtion_€",
        "solar_production_€",
        "lost_€",
    ]
].resample("1h").sum()

,consumed_€,bought_€,solar_consumtion_€,solar_production_€,lost_€
2024-04-18 00:00:00+00:00,0.005973,0.005907,0.000066,0.000066,0.0
2024-04-18 01:00:00+00:00,0.005692,0.005604,0.000088,0.000088,0.0
2024-04-18 02:00:00+00:00,0.004569,0.004487,0.000083,0.000083,0.0
2024-04-18 03:00:00+00:00,0.004619,0.004537,0.000082,0.000082,0.0
2024-04-18 04:00:00+00:00,0.004862,0.004782,0.00008,0.00008,0.0
2024-04-18 05:00:00+00:00,0.004,0.003907,0.000094,0.000094,0.0
2024-04-18 06:00:00+00:00,0.004232,0.00325,0.000982,0.000982,0.0
2024-04-18 07:00:00+00:00,0.013961,0.003512,0.010449,0.01078,0.000331
2024-04-18 08:00:00+00:00,0.020707,0.009601,0.011105,0.013326,0.002221
2024-04-18 09:00:00+00:00,0.025318,0.014264,0.011053,0.014963,0.00391


In [8]:
costs_df[
    [
        "consumed_€",
        "bought_€",
        "solar_consumtion_€",
        "solar_production_€",
        "lost_€",
    ]
].resample("1D").sum()

,consumed_€,bought_€,solar_consumtion_€,solar_production_€,lost_€
2024-04-18 00:00:00+00:00,0.314768,0.212302,0.102466,0.153052,0.050586
2024-04-19 00:00:00+00:00,0.04622,0.045577,0.000643,0.000643,0.0
